<a href="https://colab.research.google.com/github/Raph33AI/SonicSense-AI/blob/main/SonicSense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F # import torch.nn.functional
import torch.optim as optim
from torchvision import datasets, transforms

# 1. Définir le modèle (CNN simple)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Couches du réseau...
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        # Propagation avant...
        x = torch.relu(torch.max_pool2d(self.conv1(x), 2))
        x = torch.relu(torch.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = torch.relu(self.fc1(x))
        x = F.dropout(x, training=self.training) # Use F.dropout
        x = self.fc2(x)
        return torch.log_softmax(x, dim=1)

# 2. Charger les données (exemple MNIST)
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=False)

# 3. Instancier le modèle, la fonction de coût et l'optimiseur
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Entraîner le modèle
for epoch in range(10):  # Nombre d'époques
    for batch_idx, (data, target) in enumerate(train_loader):
        # Entraînement...
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

# 5. Évaluer le modèle
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        # Évaluation...
        output = model(data)
        _, predicted = torch.max(output, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = 100 * correct / total
print('Accuracy: {}'.format(accuracy))

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.331071
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.874986
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.382270
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.489720
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.640290
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.439420
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.229482
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.177342
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.350996
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.405223
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.251574
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.306478
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.203753
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.177620
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.228850
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.269507
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.198401
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.187443
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.195116
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.235656
Train Epoch:

In [28]:
import requests
import base64

# Vos identifiants d'application
CLIENT_ID = "dffbe0f471154691b105df32348a2378"
CLIENT_SECRET = "cd114e5ad1654451a84a1d3b9e32c7ee"

# Fonction pour obtenir l'access token Spotify
def get_access_token():
    auth_string = f"{CLIENT_ID}:{CLIENT_SECRET}"
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    token_url = "https://accounts.spotify.com/api/token"
    token_data = {
        "grant_type": "client_credentials"
    }
    token_headers = {
        "Authorization": f"Basic {auth_base64}"
    }

    token_response = requests.post(token_url, data=token_data, headers=token_headers)

    if token_response.status_code == 200:
        return token_response.json()["access_token"]
    else:
        print(f"Erreur lors de la récupération du token: {token_response.status_code}")
        return None

# Obtenir l'access token
ACCESS_TOKEN = get_access_token()

if ACCESS_TOKEN:
    headers = {
        "Authorization": f"Bearer {ACCESS_TOKEN}"
    }

    search_url = "https://api.spotify.com/v1/search"
    search_params = {
        "q": "genre:house",
        "type": "track",
        "limit": 5
    }
    search_response = requests.get(search_url, headers=headers, params=search_params)

    if search_response.status_code == 200:
        search_data = search_response.json()
        tracks = search_data["tracks"]["items"]

        print("Artiste - Album - Date - Tempo - Danceability - Energy - Valence - Lien")
        print("---------------------------------------------------------------------")

        for track in tracks:
            # ... (Code pour récupérer les informations de base du morceau)

            audio_features_url = f"https://api.spotify.com/v1/audio-features/{track_id}"
            audio_features_response = requests.get(audio_features_url, headers=headers)

            if audio_features_response.status_code == 200:
                audio_features = audio_features_response.json()
                tempo = audio_features.get("tempo", "N/A")
                danceability = audio_features.get("danceability", "N/A")
                energy = audio_features.get("energy", "N/A")
                valence = audio_features.get("valence", "N/A")

                print(
                    f"{artists} - {album} - {release_date} - "
                    f"{tempo:.2f} BPM - {danceability:.2f} - "
                    f"{energy:.2f} - {valence:.2f} - "
                    f"{track['external_urls']['spotify']}"
                )
            else:
                # Gestion d'erreur pour audio_features_response
                print(f"Erreur lors de la récupération des caractéristiques audio : {audio_features_response.status_code}")
    else:
        # Gestion d'erreur pour search_response
        print(f"Erreur lors de la recherche de morceaux : {search_response.status_code}")

Artiste - Album - Date - Tempo - Danceability - Energy - Valence - Lien
---------------------------------------------------------------------
Travis Scott - HIGHEST IN THE ROOM - 2019-10-04 - 76.47 BPM - 0.60 - 0.43 - 0.06 - https://open.spotify.com/track/42VsgItocQwOQC3XWZ8JNA
Travis Scott - HIGHEST IN THE ROOM - 2019-10-04 - 76.47 BPM - 0.60 - 0.43 - 0.06 - https://open.spotify.com/track/3FU6urUVsgXa6RBuV2PdRk
Travis Scott - HIGHEST IN THE ROOM - 2019-10-04 - 76.47 BPM - 0.60 - 0.43 - 0.06 - https://open.spotify.com/track/4pkb8SbRGeHAvdb87v9rpf
Travis Scott - HIGHEST IN THE ROOM - 2019-10-04 - 76.47 BPM - 0.60 - 0.43 - 0.06 - https://open.spotify.com/track/6wsqVwoiVH2kde4k4KKAFU
Travis Scott - HIGHEST IN THE ROOM - 2019-10-04 - 76.47 BPM - 0.60 - 0.43 - 0.06 - https://open.spotify.com/track/3eekarcy7kvN4yt5ZFzltW
